<a href="https://colab.research.google.com/github/1337Rinz/AI_ML/blob/main/lab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.9/491.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.9/440.9 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.0 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from dataclasses import dataclass
import pandas as pd
import numpy as np
import glob
import re
from pprint import pprint

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [3]:
class Config:
  MAX_LEN = 256
  BATCH_SIZE = 32
  LR = 0.001
  VOCAB_SIZE = 30000
  EMBED_DIM = 120
  NUM_HEAD = 8
  FF_DIM = 128
  NUM_LAYERS = 1

config = Config()

In [4]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  58.1M      0  0:00:01  0:00:01 --:--:-- 58.1M


In [5]:
!tar -xf /content/aclImdb_v1.tar.gz

In [6]:
def get_text_list_from_files(files):
  text_list = []
  for name in files:
    with open(name) as f:
      for line in f:
        text_list.append(line)
  return text_list


def get_data_from_text_files(folder_name):
  pos_files = glob.glob("aclImdb/" + folder_name + "/pos/*.txt")
  pos_texts = get_text_list_from_files(pos_files)

  neg_files = glob.glob("aclImdb/" + folder_name + "/neg/*.txt")
  neg_texts = get_text_list_from_files(neg_files)

  df = pd.DataFrame({
      "review": pos_texts + neg_texts,
      "sentiment": [0] * len(pos_texts) + [1] * len(neg_texts)
  })

  df = df.sample(len(df)).reset_index(drop=True)
  return df

In [7]:
train_df = get_data_from_text_files("train")
test_df = get_data_from_text_files("test")

all_data = train_df.append(test_df)

<ipython-input-7-8d9aac150f9d>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = train_df.append(test_df)


In [8]:
all_data

,review,sentiment
0,I decided to watch this one because it's been ...,1
1,"Mesmerizing, breathtaking and horrifying, this...",0
2,I really hope that Concorde/New Horizons wasn'...,1
3,This almost unknown gem was based on a French ...,0
4,Aside from the horrendous acting and the ridic...,1
...,...,...
24995,I think this movie is underrated. To me it fel...,0
24996,Tyra Banks needs to teach these girls that it'...,1
24997,This movie was difficult for me to watch. Stan...,1
24998,"I kid you not. Yes, ""Who's That Girl"" has the ...",0


In [9]:
def custom_standarization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
  return tf.strings.regex_replace(stripped_html, "[%s]" % re.escape("!#$%&'()*+,-./:;<=>?@\^_`{|}~"), "")

def get_vectorize_layer(texts, vocab_size, max_seq, special_tokens=["[MASK]"]):
  vectorize_layer = TextVectorization(
      max_tokens=vocab_size,
      output_mode="int",
      standardize=custom_standarization,
      output_sequence_length=max_seq,
  )
  vectorize_layer.adapt(texts)

  vocab = vectorize_layer.get_vocabulary()
  vocab = vocab[2 : vocab_size - len(special_tokens)] + ["[mask]"]
  vectorize_layer.set_vocabulary(vocab)
  return vectorize_layer


vectorize_layer = get_vectorize_layer(
    all_data.review.values.tolist(),
    config.VOCAB_SIZE,
    config.MAX_LEN,
    special_tokens=["[mask]"]
)

mask_token_id = vectorize_layer(["[mask]"]).numpy()[0][0]


In [10]:
def encode(texts):
  encoded_texts = vectorize_layer(texts)
  return encoded_texts.numpy()

def get_masked_input_and_labels(encoded_texts):
  inp_mask = np.random.rand(*encoded_texts.shape) < 0.15
  inp_mask[encoded_texts <= 2 ] = False

  labels = -1 * np.ones(encoded_texts.shape, dtype=int)
  labels[inp_mask] = encoded_texts[inp_mask]

  encoded_texts_masked = np.copy(encoded_texts)

  inp_mask_2mask = inp_mask & (np.random.rand(*encoded_texts.shape) < 0.90)
  encoded_texts_masked[inp_mask_2mask] = mask_token_id

  inp_mask_2random = inp_mask_2mask & (np.random.rand(*encoded_texts.shape)< 1/9)
  encoded_texts_masked[inp_mask_2random] = np.random.randint(
      3, mask_token_id, inp_mask_2random.sum()
  )
  sample_weights = np.ones(labels.shape)
  sample_weights[labels == -1 ] = 0

  y_labels = np.copy(encoded_texts)
  return encoded_texts_masked, y_labels, sample_weights

In [11]:
x_train = encode(train_df.review.values)
y_train = train_df.sentiment.values
train_classifier_ds = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(1000)
    .batch(config.BATCH_SIZE)
)

In [12]:
x_test = encode(test_df.review.values)
y_test = test_df.sentiment.values
test_classifier_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(config.BATCH_SIZE)
test_raw_classifier_ds = tf.data.Dataset.from_tensor_slices((test_df.review.values, y_test)).batch(config.BATCH_SIZE)

In [13]:
x_all_review = encode(all_data.review.values)
x_masked_train, y_masked_labels, sample_weights = get_masked_input_and_labels(
    x_all_review
)

mlm_ds = tf.data.Dataset.from_tensor_slices(
    (x_masked_train, y_masked_labels, sample_weights)
)
mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE)

In [14]:
def bert_module(query, key, value, i):
  attention_output = layers.MultiHeadAttention(
      num_heads=config.NUM_HEAD,
      key_dim=config.EMBED_DIM // config.NUM_HEAD,
      name="encoder_{}/multiheadattention".format(i),
  )(query, key, value)
  attention_output = layers.Dropout(0.1, name="encoder_{}/att_dropout".format(i))(attention_output)
  attention_output = layers.LayerNormalization(
      epsilon=1e-6, name="encoder_{}/att_layernormalization".format(i)
  )(query + attention_output)

  ffn = keras.Sequential(
      [
          layers.Dense(config.FF_DIM, activation="relu"),
          layers.Dense(config.EMBED_DIM),
      ],
      name="encoder_{}/ffn".format(i),
  )

  ffn_output = ffn(attention_output)
  ffn_output = layers.Dropout(0.1, name="encoder_{}/ffn_dropout".format(i))(ffn_output)
  sequence_output = layers.LayerNormalization(
      epsilon=1e-6, name="encoder_{}/ffn_layernormalization".format(i)
  )(attention_output+ffn_output)
  return sequence_output

In [15]:
def get_pos_encoding_matrix(max_len, d_emb):
  pos_enc = np.array(
      [
          [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
        if pos != 0
        else np.zeros(d_emb)
        for pos in range(max_len)
      ]
  )
  pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])
  pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])
  return pos_enc

loss_fn = keras.losses.SparseCategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.NONE
)

loss_tracker = tf.keras.metrics.Mean(name="loss")

class MaskedLanguageModel(tf.keras.Model):
  def train_step(self, inputs):
    if len(inputs) == 3:
      features, labels, sample_weight = inputs
    else:
      features, labels = inputs
      sample_weight = None

    with tf.GradientTape() as tape:
      predictions = self(features, training=True)
      loss = loss_fn(labels, predictions, sample_weight=sample_weight)

      trainable_vars = self.trainable_variables
      gradients = tape.gradient(loss, trainable_vars)

      self.optimizer.apply_gradients(zip(gradients, trainable_vars))
      loss_tracker.update_state(loss, sample_weight=sample_weight)

      return {"loss": loss_tracker.result()}


  @property
  def metrics(self):
    return [loss_tracker]

def create_masked_language_bert_model():
  inputs = layers.Input((config.MAX_LEN), dtype=tf.int64)

  word_embeddings = layers.Embedding(
      config.VOCAB_SIZE, config.EMBED_DIM, name="word_embedding"
  )(inputs)
  position_embeddings = layers.Embedding(
      input_dim=config.MAX_LEN,
      output_dim=config.EMBED_DIM,
      weights=[get_pos_encoding_matrix(config.MAX_LEN, config.EMBED_DIM)],
      name="position_embedding"
  )(tf.range(start=0, limit=config.MAX_LEN, delta=1))
  embeddings = word_embeddings + position_embeddings

  encoder_output = embeddings
  for i in range(config.NUM_LAYERS):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, i)
    mlm_output = layers.Dense(config.VOCAB_SIZE, name="mlm_cls", activation="softmax")(encoder_output)
    mlm_model = MaskedLanguageModel(inputs, mlm_output, name="masked_bert_model")

    optimizer = keras.optimizers.Adam(learning_rate=config.LR)
    mlm_model.compile(optimizer=optimizer)
    return mlm_model

In [16]:
id2token = dict(enumerate(vectorize_layer.get_vocabulary()))
token2id = {y:x for x, y in id2token.items()}

In [17]:
class MaskedTextGenerator(keras.callbacks.Callback):
  def __init__(self, sample_tokens, top_k=5):
    self.sample_tokens = sample_tokens
    self.k = top_k

  def decode(self, tokens):
    return " ".join([id2token[t] for t in tokens if t !=0])

  def convert_ids_to_tokens(self, id):
    return id2token[id]

  def on_epoch_end(self, epoch, logs=None):
    prediction = self.model.predict(self.sample_tokens)

    masked_index = np.where(self.sample_tokens == mask_token_id)
    masked_index = masked_index[1]
    mask_prediction = prediction[0][masked_index]

    top_indices = masked_index[0].argsort()[-self.k :][::-1]
    values = mask_prediction[0][top_indices]

    for i in range(len(top_indices)):
      p = top_indices[i]
      v = values[i]

      tokens = np.copy(sample_tokens[0])
      tokens[masked_index[0]] = p
      result = {
          "input_text": self.decode(sample_tokens[0].numpy()),
          "prediction": self.decode(tokens),
          "probability": v,
          "predicted mask token": self.convert_ids_to_tokens(p),
      }
      pprint(result)


In [18]:
sample_tokens = vectorize_layer(["I have watched this [mask] and it was awesome"])
generator_callback = MaskedTextGenerator(sample_tokens.numpy())

bert_masked_model = create_masked_language_bert_model()
bert_masked_model.summary()


Model: "masked_bert_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256)]                0         []                            
                                                                                                  
 word_embedding (Embedding)  (None, 256, 120)             3600000   ['input_1[0][0]']             
                                                                                                  
 tf.__operators__.add (TFOp  (None, 256, 120)             0         ['word_embedding[0][0]']      
 Lambda)                                                                                          
                                                                                                  
 encoder_0/multiheadattenti  (None, 256, 120)             58080     ['tf.__operato

In [19]:
bert_masked_model.fit(mlm_ds, epochs=1, callbacks=[generator_callback])
bert_masked_model.save("bert_mlm_imdb.h5")

1/1 [==============================] - 0s 224ms/step
{'input_text': 'i have watched this [mask] and it was awesome',
 'predicted mask token': '',
 'prediction': 'i have watched this and it was awesome',
 'probability': 3.8476543e-08}
1563/1563 [==============================] - 283s 174ms/step - loss: 6.9982


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
mlm_model = keras.models.load_model("bert_mlm_imdb.h5", custom_objects={"MaskedLanguageModel": MaskedLanguageModel})
pretrained_bert_model = tf.keras.Model(
    mlm_model.input,
    mlm_model.get_layer("encoder_0/ffn_layernormalization").output
)

pretrained_bert_model.trainable = False

def create_classifier_bert_model():
  inputs = layers.Input((config.MAX_LEN), dtype=tf.int64)
  sequence_output = pretrained_bert_model(inputs)
  pooled_output = layers.GlobalMaxPooling1D()(sequence_output)
  hidden_layer = layers.Dense(64, activation="relu")(pooled_output)

  outputs = layers.Dense(1, activation="sigmoid")(hidden_layer)
  classifier_model = keras.Model(inputs, outputs, name="classification")
  optimizer = keras.optimizers.Adam()
  classifier_model.compile(
      optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy"]
  )
  return classifier_model


In [21]:
classifier_model = create_classifier_bert_model()
classifier_model.summary()

Model: "classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256)]             0         
                                                                 
 model (Functional)          (None, 256, 120)          3689528   
                                                                 
 global_max_pooling1d (Glob  (None, 120)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense_2 (Dense)             (None, 64)                7744      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3697337 (14.10 MB)
Trainable params: 7809 (30.50 KB)
Non-trainable params: 3689528 (14.07 MB)
__________

In [22]:
classifier_model.fit(
    train_classifier_ds,
    epochs=5,
    validation_data=test_classifier_ds,
)

Epoch 1/5
782/782 [==============================] - 16s 18ms/step - loss: 0.6982 - accuracy: 0.5667 - val_loss: 0.6571 - val_accuracy: 0.6121
Epoch 2/5
782/782 [==============================] - 17s 22ms/step - loss: 0.6632 - accuracy: 0.6065 - val_loss: 0.7241 - val_accuracy: 0.5459
Epoch 3/5
782/782 [==============================] - 13s 17ms/step - loss: 0.6613 - accuracy: 0.6089 - val_loss: 0.6420 - val_accuracy: 0.6318
Epoch 4/5
782/782 [==============================] - 14s 17ms/step - loss: 0.6521 - accuracy: 0.6180 - val_loss: 0.6809 - val_accuracy: 0.5798
Epoch 5/5
782/782 [==============================] - 13s 16ms/step - loss: 0.6516 - accuracy: 0.6184 - val_loss: 0.6574 - val_accuracy: 0.6084


In [23]:
def get_end_to_end(model):
  inputs_string = keras.Input(shape=(1,), dtype="string")
  indices = vectorize_layer(inputs_string)
  outputs = model(indices)
  end_to_end_model = keras.Model(inputs_string, outputs, name="end_to_end_model")

  optimizer = keras.optimizers.Adam(learning_rate=config.LR)
  end_to_end_model.compile(
      optimizer=optimizer,
      loss="binary_crossentropy",
      metrics=["accuracy"]
  )
  return end_to_end_model

In [24]:
end_to_end_classification_model = get_end_to_end(classifier_model)
end_to_end_classification_model.evaluate(test_raw_classifier_ds)


782/782 [==============================] - 10s 11ms/step - loss: 0.6574 - accuracy: 0.6084


[0.6573957800865173, 0.6083999872207642]